# 上級編1

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 課題の生成行列
G = np.matrix([
    [1, 0, 1, 0, 1, 1],
    [1, 1, 1, 1, 0, 1],
    [0, 1, 1, 0, 0, 1]
], dtype="u1")

# G = np.matrix([
#     [1, 1, 0, 0, 1, 0, 0],
#     [1, 0, 1, 0, 0, 1, 0],
#     [1, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 0, 1]
# ], dtype="u1")

k, n = G.shape
print(G)
k, n

[[1 0 1 0 1 1]
 [1 1 1 1 0 1]
 [0 1 1 0 0 1]]


(3, 6)

## 掃き出し法で組織符号の生成行列に変換

In [3]:
k = G.shape[0]

A = np.matrix(np.identity(k), dtype="u1")
G_src = G.copy()[:,:k]
# print(G_src)
for i in range(k):
    col_i = np.array(G_src)[:,i]
    if col_i[i] == 1:
        A_i = np.identity(k)
        A_i[:,i] = col_i
        A_i = np.matrix(A_i, dtype="u1")
        A = A_i * A % 2
        # print(f"A_i=\n{A_i}")
        # print(f"A=\n{A}")
        G_src = A_i * G_src % 2
        # print(f"掃き出し{i + 1}回目")
        # print(G_src)

Tissue_G = A * G % 2
Tissue_G

matrix([[1, 0, 0, 1, 0, 0],
        [0, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 1]], dtype=uint8)

## 標準アレイ

In [4]:
all_src_ex0 = np.matrix([[int(j) for j in format(i, f"0{k}b")] for i in range(1, 2 ** k)], dtype="u1")
all_code_ex0 = all_src_ex0 * Tissue_G % 2
all_code_ex0_str = ["".join(str(i) for i in l) for l in all_code_ex0.tolist()]
all_code_ex0_str

['001111', '010110', '011001', '100100', '101011', '110010', '111101']

In [5]:
# 1ビットエラー全パターン?
# all_error_str = [format(2 ** i, f"0{n}b") for i in range(n)]
# 適当に追加
# all_error_str += ["000011"]
# 起こりやすい誤りを勝手に定義
all_error_str = ["000001", "000010", "000100", "001000", "010000", "000011", "000101"]
all_error = np.matrix([[int(j) for j in moji] for moji in all_error_str])
all_error_str

['000001', '000010', '000100', '001000', '010000', '000011', '000101']

In [50]:
std_arr_matrix = [[np.matrix((e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
# std_arr_values = [["".join(str(k) for k in (e + w) % 2) for w in np.array(all_code_ex0)] for e in np.array(all_error)]
std_arr_values = [["".join(str(k) for k in j.tolist()[0]) for j in i] for i in std_arr_matrix]
# np.array(std_arr_values)
std_arr_df = pd.DataFrame(std_arr_values, columns=all_code_ex0_str, index=all_error_str)
std_arr_df

,001111,010110,011001,100100,101011,110010,111101
000001,001110,010111,011000,100101,101010,110011,111100
000010,001101,010100,011011,100110,101001,110000,111111
000100,001011,010010,011101,100000,101111,110110,111001
001000,000111,011110,010001,101100,100011,111010,110101
010000,011111,000110,001001,110100,111011,100010,101101
000011,001100,010101,011010,100111,101000,110001,111110
000101,001010,010011,011100,100001,101110,110111,111000


In [36]:
# 重複がないか確認
len(set(std_arr_df.values.ravel()))

49

## 各受信後のシンドローム計算

In [37]:
# パリティ生成部分
P = Tissue_G[:, k:]
# print(P)
# 検査行列
H = np.concatenate([P.T, np.identity(k, dtype=np.uint8)], 1)
H

matrix([[1, 1, 1, 1, 0, 0],
        [0, 1, 1, 0, 1, 0],
        [0, 0, 1, 0, 0, 1]], dtype=uint8)

In [57]:
syndrome_values = [["".join(str(k) for k in (j * H.T % 2).tolist()[0]) for j in i] for i in std_arr_matrix]
syndrome_df = pd.DataFrame(syndrome_values, columns=all_code_ex0_str, index=all_error_str)
syndrome_df

,001111,010110,011001,100100,101011,110010,111101
000001,001,001,001,001,001,001,001
000010,010,010,010,010,010,010,010
000100,100,100,100,100,100,100,100
001000,111,111,111,111,111,111,111
010000,110,110,110,110,110,110,110
000011,011,011,011,011,011,011,011
000101,101,101,101,101,101,101,101
